# Ensemble Techniques

## Environment Setup

### Imports

In [1]:
import os
os.sys.path.append("utils")

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from monai.transforms import AsDiscrete
from utils.Inference import ensemble_inference
from sklearn.linear_model import LogisticRegression

### Config

In [2]:
seed = 33
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

## Load Train Data

In [3]:
# Load train data
subject_ids = pd.read_csv('./data/TRAIN.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'./outputs/AHNet/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'./outputs/UNet/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'./outputs/SegResNet/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'./outputs/UNETR/pred_segs/train_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'./outputs/gt_segs/train_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
train_df = pd.DataFrame()
train_df['SubjectID'] = subject_ids
train_df['AHNet'] = ah_segs
train_df['UNet'] = unet_segs
train_df['SegResNet'] = segresnet_segs
train_df['UNETR'] = untr_segs
train_df['GT'] = gt_segs

train_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100381A,"[./outputs/AHNet/pred_segs/train_pred_segs/pred_100381A_TC.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100381A_WT.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[./outputs/UNet/pred_segs/train_pred_segs/pred_100381A_TC.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100381A_WT.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[./outputs/SegResNet/pred_segs/train_pred_segs/pred_100381A_TC.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100381A_WT.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[./outputs/UNETR/pred_segs/train_pred_segs/pred_100381A_TC.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100381A_WT.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100381A_ET.npz]","[./outputs/gt_segs/train_gt_segs/gt_100381A_TC.npz, ./outputs/gt_segs/train_gt_segs/gt_100381A_WT.npz, ./outputs/gt_segs/train_gt_segs/gt_100381A_ET.npz]"
1,100414B,"[./outputs/AHNet/pred_segs/train_pred_segs/pred_100414B_TC.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100414B_WT.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[./outputs/UNet/pred_segs/train_pred_segs/pred_100414B_TC.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100414B_WT.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[./outputs/SegResNet/pred_segs/train_pred_segs/pred_100414B_TC.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100414B_WT.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[./outputs/UNETR/pred_segs/train_pred_segs/pred_100414B_TC.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100414B_WT.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100414B_ET.npz]","[./outputs/gt_segs/train_gt_segs/gt_100414B_TC.npz, ./outputs/gt_segs/train_gt_segs/gt_100414B_WT.npz, ./outputs/gt_segs/train_gt_segs/gt_100414B_ET.npz]"
2,100132B,"[./outputs/AHNet/pred_segs/train_pred_segs/pred_100132B_TC.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100132B_WT.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[./outputs/UNet/pred_segs/train_pred_segs/pred_100132B_TC.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100132B_WT.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[./outputs/SegResNet/pred_segs/train_pred_segs/pred_100132B_TC.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100132B_WT.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[./outputs/UNETR/pred_segs/train_pred_segs/pred_100132B_TC.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100132B_WT.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100132B_ET.npz]","[./outputs/gt_segs/train_gt_segs/gt_100132B_TC.npz, ./outputs/gt_segs/train_gt_segs/gt_100132B_WT.npz, ./outputs/gt_segs/train_gt_segs/gt_100132B_ET.npz]"
3,100212A,"[./outputs/AHNet/pred_segs/train_pred_segs/pred_100212A_TC.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100212A_WT.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[./outputs/UNet/pred_segs/train_pred_segs/pred_100212A_TC.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100212A_WT.npz, ./outputs/UNet/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[./outputs/SegResNet/pred_segs/train_pred_segs/pred_100212A_TC.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100212A_WT.npz, ./outputs/SegResNet/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[./outputs/UNETR/pred_segs/train_pred_segs/pred_100212A_TC.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100212A_WT.npz, ./outputs/UNETR/pred_segs/train_pred_segs/pred_100212A_ET.npz]","[./outputs/gt_segs/train_gt_segs/gt_100212A_TC.npz, ./outputs/gt_segs/train_gt_segs/gt_100212A_WT.npz, ./outputs/gt_segs/train_gt_segs/gt_100212A_ET.npz]"
4,100243B,"[./outputs/AHNet/pred_segs/train_pred_segs/pred_100243B_TC.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_100243B_WT.npz, ./outputs/AHNet/pred_segs/train_pred_segs/pred_10

## Load Test Data

In [4]:
subject_ids = pd.read_csv('./data/TEST.csv')['SubjectID'].values

ah_segs, unet_segs, segresnet_segs, untr_segs, gt_segs = [], [], [], [], []
for sid in subject_ids:
    ah_channels, unet_channels, segresnet_channels, untr_channels, gt_channels = [], [], [], [], []
    for channel in ['TC', 'WT', 'ET']:
        ah_channels.append(f'./outputs/AHNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        unet_channels.append(f'./outputs/UNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        segresnet_channels.append(f'./outputs/SegResNet/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        untr_channels.append(f'./outputs/UNETR/pred_segs/test_pred_segs/pred_{sid}_{channel}.npz')
        gt_channels.append(f'./outputs/gt_segs/test_gt_segs/gt_{sid}_{channel}.npz')
    
    ah_segs.append(ah_channels)
    unet_segs.append(unet_channels)
    segresnet_segs.append(segresnet_channels)
    untr_segs.append(untr_channels)
    gt_segs.append(gt_channels)

# Dataframe
test_df = pd.DataFrame()
test_df['SubjectID'] = subject_ids
test_df['AHNet'] = ah_segs
test_df['UNet'] = unet_segs
test_df['SegResNet'] = segresnet_segs
test_df['UNETR'] = untr_segs
test_df['GT'] = gt_segs

test_df.head()

,SubjectID,AHNet,UNet,SegResNet,UNETR,GT
0,100214B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100214B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100214B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100214B_ET.npz]"
1,100340A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100340A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100340A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100340A_ET.npz]"
2,100391A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100391A_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100391A_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100391A_ET.npz]"
3,100190B,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/SegResNet/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_TC.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_WT.npz, ./outputs/UNETR/pred_segs/test_pred_segs/pred_100190B_ET.npz]","[./outputs/gt_segs/test_gt_segs/gt_100190B_TC.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_WT.npz, ./outputs/gt_segs/test_gt_segs/gt_100190B_ET.npz]"
4,100142A,"[./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_TC.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_WT.npz, ./outputs/AHNet/pred_segs/test_pred_segs/pred_100142A_ET.npz]","[./outputs/UNet/pred_segs/test_pred_segs/pred_1

## Ensemble 1 - Average

In [5]:
def avg(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.mean(stacked_predictions, dim=0)

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    avg_df = ensemble_inference(test_df, avg, threshold)
    mean = avg_df['Dice'].mean()
    print(f'Average Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = avg_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/avg_test_scores.csv', index=False)
df.describe()

Average Mean Dice for threshold 0.1: 0.5150403138250113
Average Mean Dice for threshold 0.2: 0.5272680526043498
Average Mean Dice for threshold 0.3: 0.6201561954953978
Average Mean Dice for threshold 0.4: 0.6306360651919197
Average Mean Dice for threshold 0.5: 0.6117088165014021
Average Mean Dice for threshold 0.6: 0.582749746499523
Average Mean Dice for threshold 0.7: 0.5514685651587863
Average Mean Dice for threshold 0.8: 0.4944665648283497
Average Mean Dice for threshold 0.9: 0.44803049511486487
BEST THRESHOLD: 0.4


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.630636,0.662461,0.673749,0.533071,6.677419,6.516129,87.129032,8.709677,8.709677,5.129032,16592.322581,16591.774194,14575.709677,17105.387097,17985.870968,13810.806452
std,0.259768,0.255310,0.260801,0.293511,5.906985,5.977017,166.985177,9.103822,9.103822,4.951376,24168.923954,24207.681579,22133.776319,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,7.000000,6.000000,0.000000,44.000000,44.000000,0.000000
25%,0.484863,0.554968,0.560128,0.282297,3.000000,2.500000,5.500000,2.000000,2.000000,1.500000,1057.000000,1048.000000,662.500000,802.000000,802.000000,410.000000
50%,0.676840,0.717632,0.698113,0.542388,5.000000,5.000000,14.000000,6.000000,6.000000,4.000000,4993.000000,4986.000000,3298.000000,3847.000000,4138.000000,1944.000000
75%,0.842653,0.869613,0.885937,0.769723,8.000000,8.000000,65.000000,13.500000,13.500000,7.000000,29342.500000,29308.000000,27872.000000,26305.000000,27036.500000,23520.500000
max,0.946255,0.955846,0.958104,0.924816,25.000000,25.000000,684.000000,43.000000,43.000000,21.000000,90746.000000,90706.000000,83437.000000,102770.000000,105729.000000,92221.000000


## Ensemble 2 - Weighted Average (Mean Dice Weights)

In [ ]:
def weigthed_avg(predictions):

    # Weights form validation set AHNet, SegResNet, UNet, UNETR
    weights = np.array([0.546402, 0.648790, 0.428639, 0.547957])
    weights = weights / np.sum(weights)
    weights = torch.tensor(weights).float()
    # Weighted average
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.sum(stacked_predictions * weights[:, None, None, None, None, None], dim=0)
     
#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    weigthed_avg_df = ensemble_inference(test_df, weigthed_avg, threshold)
    mean = weigthed_avg_df['Dice'].mean()
    print(f'Weighted Average Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = weigthed_avg_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/weighted_avg_test_scores.csv', index=False)
df.describe()

Weighted Average Mean Dice for threshold 0.1: 0.5145711503442256
Weighted Average Mean Dice for threshold 0.2: 0.576161464856517
Weighted Average Mean Dice for threshold 0.3: 0.632053408891924
Weighted Average Mean Dice for threshold 0.4: 0.6359004551965383
Weighted Average Mean Dice for threshold 0.5: 0.6326853892495555
Weighted Average Mean Dice for threshold 0.6: 0.5931650337673002
Weighted Average Mean Dice for threshold 0.7: 0.5561348192634121
Weighted Average Mean Dice for threshold 0.8: 0.49360962188051594
Weighted Average Mean Dice for threshold 0.9: 0.4448244151088499
BEST THRESHOLD: 0.4


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.635900,0.667215,0.679828,0.538870,6.903226,6.419355,109.774194,8.709677,8.709677,5.129032,16629.322581,16633.903226,14432.903226,17105.387097,17985.870968,13810.806452
std,0.257415,0.252058,0.256418,0.295345,5.985287,5.863868,204.315884,9.103822,9.103822,4.951376,24313.317312,24377.260244,22128.085428,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,6.000000,5.000000,0.000000,44.000000,44.000000,0.000000
25%,0.498647,0.566927,0.570827,0.290649,2.500000,2.000000,6.000000,2.000000,2.000000,1.500000,1042.000000,1032.500000,635.000000,802.000000,802.000000,410.000000
50%,0.675439,0.723906,0.727524,0.549588,5.000000,5.000000,21.000000,6.000000,6.000000,4.000000,4940.000000,4945.000000,3194.000000,3847.000000,4138.000000,1944.000000
75%,0.846335,0.871845,0.890754,0.776012,8.500000,8.000000,103.500000,13.500000,13.500000,7.000000,29476.500000,29444.500000,27817.000000,26305.000000,27036.500000,23520.500000
max,0.947493,0.956289,0.958427,0.927762,25.000000,25.000000,922.000000,43.000000,43.000000,21.000000,91540.000000,91606.000000,83508.000000,102770.000000,105729.000000,92221.000000


## Ensemble 3 - Median 

In [ ]:
def median(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.median(stacked_predictions, dim=0).values

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    median_df = ensemble_inference(test_df, median, threshold)
    mean = median_df['Dice'].mean()
    print(f'Median Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = median_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/median_test_scores.csv', index=False)
df.describe()

Median Mean Dice for threshold 0.1: 0.6100782444400172
Median Mean Dice for threshold 0.2: 0.6120831245376218
Median Mean Dice for threshold 0.3: 0.6029955754356999
Median Mean Dice for threshold 0.4: 0.592251036436327
Median Mean Dice for threshold 0.5: 0.5806126330168017
Median Mean Dice for threshold 0.6: 0.5694510761287904
Median Mean Dice for threshold 0.7: 0.5551219413117054
Median Mean Dice for threshold 0.8: 0.536392334788557
Median Mean Dice for threshold 0.9: 0.5016900248104527
BEST THRESHOLD: 0.2


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.612083,0.638850,0.649940,0.531447,6.225806,5.741935,7.032258,8.709677,8.709677,5.129032,15876.483871,16000.548387,13569.354839,17105.387097,17985.870968,13810.806452
std,0.273661,0.269515,0.274134,0.306581,5.948163,5.483112,8.121920,9.103822,9.103822,4.951376,24037.063603,24258.577521,21550.894202,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.486434,0.508295,0.520046,0.242854,2.500000,2.000000,2.000000,2.000000,2.000000,1.500000,943.500000,937.500000,592.000000,802.000000,802.000000,410.000000
50%,0.635331,0.679783,0.720000,0.571091,4.000000,4.000000,4.000000,6.000000,6.000000,4.000000,3859.000000,3888.000000,2995.000000,3847.000000,4138.000000,1944.000000
75%,0.850039,0.868642,0.884660,0.785967,8.000000,7.500000,9.000000,13.500000,13.500000,7.000000,27416.000000,27688.000000,25957.500000,26305.000000,27036.500000,23520.500000
max,0.947509,0.954614,0.956809,0.932726,25.000000,25.000000,35.000000,43.000000,43.000000,21.000000,90012.000000,90697.000000,81341.000000,102770.000000,105729.000000,92221.000000


## Ensemble 4 - Maximum Confidence 

In [ ]:
def max_confidence(predictions):
    stacked_predictions = torch.stack(predictions, dim=0)
    return torch.max(stacked_predictions, dim=0).values

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    max_confidence_df = ensemble_inference(test_df, max_confidence, threshold)
    mean = max_confidence_df['Dice'].mean()
    print(f'Max Confidence Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = max_confidence_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/max_confidence_test_scores.csv', index=False)
df.describe()

Max Confidence Mean Dice for threshold 0.1: 0.4769555576325905
Max Confidence Mean Dice for threshold 0.2: 0.5018590381669421
Max Confidence Mean Dice for threshold 0.3: 0.5132624893899886
Max Confidence Mean Dice for threshold 0.4: 0.5196868994032904
Max Confidence Mean Dice for threshold 0.5: 0.5234780021372342
Max Confidence Mean Dice for threshold 0.6: 0.5252501968624851
Max Confidence Mean Dice for threshold 0.7: 0.5257121397167324
Max Confidence Mean Dice for threshold 0.8: 0.5242511283835366
Max Confidence Mean Dice for threshold 0.9: 0.5196892056584118
BEST THRESHOLD: 0.7


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.525712,0.563654,0.575627,0.437288,25.709677,21.967742,37.870968,8.709677,8.709677,5.129032,19678.903226,19567.967742,19298.677419,17105.387097,17985.870968,13810.806452
std,0.318874,0.324253,0.333119,0.309278,43.836205,36.408318,55.675094,9.103822,9.103822,4.951376,25807.250693,25731.658040,25660.328382,25565.209783,26912.616906,22937.645993
min,0.002576,0.005687,0.002040,0.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.000000,1821.000000,1813.000000,1750.000000,44.000000,44.000000,0.000000
25%,0.218996,0.249058,0.249256,0.133651,8.500000,7.000000,10.000000,2.000000,2.000000,1.500000,3700.500000,3664.000000,3465.500000,802.000000,802.000000,410.000000
50%,0.632381,0.661653,0.712883,0.479532,13.000000,11.000000,19.000000,6.000000,6.000000,4.000000,6310.000000,5889.000000,5875.000000,3847.000000,4138.000000,1944.000000
75%,0.794857,0.850547,0.873580,0.694644,22.000000,19.500000,30.000000,13.500000,13.500000,7.000000,32187.000000,32010.000000,32388.000000,26305.000000,27036.500000,23520.500000
max,0.932704,0.950421,0.953007,0.894683,221.000000,190.000000,251.000000,43.000000,43.000000,21.000000,99301.000000,98700.000000,97619.000000,102770.000000,105729.000000,92221.000000


## Ensemble 5 - Majority Voting

In [ ]:
def majority_voting(predictions):
    transforms =[AsDiscrete(threshold=0.3), AsDiscrete(threshold=0.5), AsDiscrete(threshold=0.2), AsDiscrete(threshold=0.4)]
    predictions = [transform(prediction) for transform, prediction in zip(transforms, predictions)]
    stacked_predictions = torch.stack(predictions, dim=0)  
    return torch.mode(stacked_predictions, dim=0).values

#---------------------------------------------------------#

majority_vote_df = ensemble_inference(test_df, majority_voting, threshold)
majority_vote_df.to_csv('./outputs/Ensemble/majority_vote_test_scores.csv', index=False)
majority_vote_df.describe()

,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.598177,0.629584,0.634287,0.518907,5.774194,5.322581,6.967742,8.709677,8.709677,5.129032,14518.096774,14620.096774,12023.290323,17105.387097,17985.870968,13810.806452
std,0.281068,0.273253,0.281470,0.319628,5.529977,5.042401,8.130125,9.103822,9.103822,4.951376,22096.846216,22308.181914,19524.494051,25565.209783,26912.616906,22937.645993
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,44.000000,44.000000,0.000000
25%,0.406790,0.476757,0.476810,0.194828,2.000000,2.000000,2.000000,2.000000,2.000000,1.500000,735.000000,726.500000,461.000000,802.000000,802.000000,410.000000
50%,0.654574,0.685359,0.715392,0.594832,4.000000,4.000000,6.000000,6.000000,6.000000,4.000000,3544.000000,3562.000000,2571.000000,3847.000000,4138.000000,1944.000000
75%,0.832495,0.866259,0.871473,0.795037,7.500000,7.000000,8.500000,13.500000,13.500000,7.000000,24593.000000,24801.500000,21853.000000,26305.000000,27036.500000,23520.500000
max,0.951305,0.957194,0.959260,0.937462,21.000000,21.000000,38.000000,43.000000,43.000000,21.000000,82128.000000,82342.000000,73322.000000,102770.000000,105729.000000,92221.000000


## Ensemble 6 - Logistic Regression on each Test Image

In [ ]:
def logreg_ensemble(predictions, targets):
    stacked_predictions = torch.stack(predictions, dim=0)
    num_models, batch, channels, depth, height, width = stacked_predictions.shape

    # Flatten
    flattened_predictions = stacked_predictions.reshape(num_models, -1).T 
    flattened_targets = targets.reshape(-1)
    
    # Logistic Regression
    meta_model = LogisticRegression(max_iter=1000)
    meta_model.fit(flattened_predictions, flattened_targets)
    ensemble_predictions = meta_model.predict_proba(flattened_predictions)
    
    # Reshape to original shape
    ensemble_predictions_reshaped = ensemble_predictions[:, 1].reshape(batch, channels, depth, height, width)
    return ensemble_predictions_reshaped

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    log_df = ensemble_inference(test_df, logreg_ensemble, threshold, True)
    mean = log_df['Dice'].mean()
    print(f'Logistic Regression Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = log_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/test_logreg_test_scores.csv', index=False)
df.describe()


Logistic Regression Mean Dice for threshold 0.1: 0.6961464016668258
Logistic Regression Mean Dice for threshold 0.2: 0.693949933494291
Logistic Regression Mean Dice for threshold 0.3: 0.6907285230775033
Logistic Regression Mean Dice for threshold 0.4: 0.6873722626797615
Logistic Regression Mean Dice for threshold 0.5: 0.6813228882909301
Logistic Regression Mean Dice for threshold 0.6: 0.670750834483413
Logistic Regression Mean Dice for threshold 0.7: 0.6421724648004578
Logistic Regression Mean Dice for threshold 0.8: 0.6066610277179749
Logistic Regression Mean Dice for threshold 0.9: 0.5064416873839593
BEST THRESHOLD: 0.1


,Dice,Dice TC,Dice WT,Dice ET,Pred NM TC,Pred NM WT,Pred NM ET,GT NM TC,GT NM WT,GT NM ET,Pred V TC,Pred V WT,Pred V ET,GT V TC,GT V WT,GT V ET
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.696146,0.729733,0.743127,0.590396,8.064516,7.064516,71.322581,8.709677,8.709677,5.129032,17306.774194,17372.709677,14472.612903,17105.387097,17985.870968,13810.806452
std,0.221089,0.199089,0.205319,0.290590,7.882197,6.757393,161.865662,9.103822,9.103822,4.951376,26300.231843,26456.874389,23840.836203,25565.209783,26912.616906,22937.645993
min,0.058690,0.108108,0.067961,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,48.000000,32.000000,0.000000,44.000000,44.000000,0.000000
25%,0.565964,0.623152,0.644902,0.434253,2.000000,2.000000,1.500000,2.000000,2.000000,1.500000,874.500000,845.500000,411.000000,802.000000,802.000000,410.000000
50%,0.771663,0.784452,0.798665,0.652569,6.000000,5.000000,6.000000,6.000000,6.000000,4.000000,3700.000000,3650.000000,2633.000000,3847.000000,4138.000000,1944.000000
75%,0.865933,0.881005,0.898048,0.810923,9.500000,9.000000,25.500000,13.500000,13.500000,7.000000,29011.000000,28953.000000,26329.000000,26305.000000,27036.500000,23520.500000
max,0.953246,0.958113,0.960400,0.941226,31.000000,29.000000,675.000000,43.000000,43.000000,21.000000,102821.000000,103018.000000,93728.000000,102770.000000,105729.000000,92221.000000


## Ensemble 7 - Logistic Regression on the Train set

In [7]:
# Find highest shape of both training and test sets
max_shape = [0, 0, 0]
for i in tqdm(range(len(train_df))):
    data = np.load(train_df['AHNet'][i][0])['arr_0']
    max_shape = np.maximum(max_shape, data.shape)
for i in tqdm(range(len(test_df))):
    data = np.load(test_df['AHNet'][i][0])['arr_0']
    max_shape = np.maximum(max_shape, data.shape)
print(max_shape)

100%|██████████| 31/31 [00:04<00:00,  6.70it/s]

[315 315 308]


In [8]:
meta_model = LogisticRegression(max_iter=1000)
for i in tqdm(range(len(train_df))):
        # Subject & Label
        subject_id = train_df['SubjectID'][i]
        load_label = train_df['GT'][i]
        
        # Images
        load_ahnet = train_df['AHNet'][i]
        load_segresnet = train_df['SegResNet'][i]
        load_unet = train_df['UNet'][i]
        load_unetr = train_df['UNETR'][i]
        
        # Params 
        image_voxel_volume = np.prod((1,1,1))
        label_voxel_volume = np.prod((1,1,1))

        # Load Images and Labels
        ahnet_image = [np.load(x)['arr_0'] for x in load_ahnet]
        segresnet_image = [np.load(x)['arr_0'] for x in load_segresnet]
        unet_image = [np.load(x)['arr_0'] for x in load_unet]
        unetr_image = [np.load(x)['arr_0'] for x in load_unetr]
        img_label = [np.load(x)['arr_0'] for x in load_label] 

        # Pad Images
        ahnet_image = [np.pad(x, ((0, max_shape[0] - x.shape[0]), (0, max_shape[1] - x.shape[1]), (0, max_shape[2] - x.shape[2])), 'constant') for x in ahnet_image]
        segresnet_image = [np.pad(x, ((0, max_shape[0] - x.shape[0]), (0, max_shape[1] - x.shape[1]), (0, max_shape[2] - x.shape[2])), 'constant') for x in segresnet_image]
        unet_image = [np.pad(x, ((0, max_shape[0] - x.shape[0]), (0, max_shape[1] - x.shape[1]), (0, max_shape[2] - x.shape[2])), 'constant') for x in unet_image]
        unetr_image = [np.pad(x, ((0, max_shape[0] - x.shape[0]), (0, max_shape[1] - x.shape[1]), (0, max_shape[2] - x.shape[2])), 'constant') for x in unetr_image]
        img_label = [np.pad(x, ((0, max_shape[0] - x.shape[0]), (0, max_shape[1] - x.shape[1]), (0, max_shape[2] - x.shape[2])), 'constant') for x in img_label]
        
        # To Tensor
        ahnet_image = [torch.from_numpy(x) for x in ahnet_image]
        segresnet_image = [torch.from_numpy(x) for x in segresnet_image]
        unet_image = [torch.from_numpy(x) for x in unet_image]
        unetr_image = [torch.from_numpy(x) for x in unetr_image]
        img_label = [torch.from_numpy(x) for x in img_label]
            
        # Stack Images and Label
        ahnet_image = torch.stack(ahnet_image, dim = 0).unsqueeze(0)
        segresnet_image = torch.stack(segresnet_image, dim = 0).unsqueeze(0)
        unet_image = torch.stack(unet_image, dim = 0).unsqueeze(0)
        unetr_image = torch.stack(unetr_image, dim = 0).unsqueeze(0)
        img_label = torch.stack(img_label, dim = 0).unsqueeze(0)

        # Prepare features
        stacked_predictions = torch.stack([ahnet_image, segresnet_image, unet_image, unetr_image], dim=0)
        num_models, batch, channels, depth, height, width = stacked_predictions.shape

        # Flatten
        flattened_predictions = stacked_predictions.reshape(num_models, -1).T 
        flattened_targets = img_label.reshape(-1)
    
        # Logistic Regression
        meta_model.fit(flattened_predictions, flattened_targets)

# Save model
torch.save(meta_model, './outputs/Ensemble/logreg.pth')

  0%|          | 1/248 [02:36<10:45:08, 156.71s/it]


KeyboardInterrupt: 

In [ ]:
load_logreg = torch.load('./outputs/Ensemble/logreg.pth')

def logreg_ensemble(predictions, model):
    stacked_predictions = torch.stack(predictions, dim=0)
    num_models, batch, channels, depth, height, width = stacked_predictions.shape
    
    # Torch pad images
    max_shape = [315, 315, 308]
    stacked_predictions = torch.nn.functional.pad(stacked_predictions, (0, 0, 0, 0, 0, 0, 0, max_shape[2] - stacked_predictions.shape[5], 0, max_shape[1] - stacked_predictions.shape[4], 0, max_shape[0] - stacked_predictions.shape[3]))
    print(stacked_predictions.shape)

    # Flatten
    flattened_predictions = stacked_predictions.reshape(num_models, -1).T 
    
    # Logistic Regression
    ensemble_predictions = model.predict_proba(flattened_predictions)
    
    # Reshape to original shape
    ensemble_predictions_reshaped = ensemble_predictions[:, 1].reshape(batch, channels, depth, height, width)
    return ensemble_predictions_reshaped

#---------------------------------------------------------#

best_mean = 0
best_threshold = 0
df = None
for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    log_df = ensemble_inference(test_df, logreg_ensemble, threshold, model = load_logreg)
    mean = log_df['Dice'].mean()
    print(f'Logistic Regression Mean Dice for threshold {threshold}: {mean}')
    if mean > best_mean:
        best_mean = mean
        best_threshold = threshold
        df = log_df
print(f'BEST THRESHOLD: {best_threshold}')
df.to_csv('./outputs/Ensemble/train_logreg_test_scores.csv', index=False)
df.describe()

torch.Size([1, 3, 220, 220, 156]) torch.Size([1, 3, 220, 220, 156])
Dice Score: 0.46267402172088623
torch.Size([1, 3, 300, 300, 146]) torch.Size([1, 3, 300, 300, 146])
Dice Score: 0.6594907641410828
torch.Size([1, 3, 220, 220, 156]) torch.Size([1, 3, 220, 220, 156])
Dice Score: 0.5897840857505798
torch.Size([1, 3, 220, 220, 156]) torch.Size([1, 3, 220, 220, 156])
Dice Score: 0.7026982307434082
torch.Size([1, 3, 220, 220, 158]) torch.Size([1, 3, 220, 220, 158])
Dice Score: 0.585826575756073
torch.Size([1, 3, 256, 256, 204]) torch.Size([1, 3, 256, 256, 204])
Dice Score: 0.05669724941253662
torch.Size([1, 3, 241, 241, 166]) torch.Size([1, 3, 241, 241, 166])
Dice Score: 0.8236450552940369
torch.Size([1, 3, 220, 220, 164]) torch.Size([1, 3, 220, 220, 164])
Dice Score: 0.7152915000915527
torch.Size([1, 3, 220, 220, 156]) torch.Size([1, 3, 220, 220, 156])
Dice Score: 0.8950082659721375
torch.Size([1, 3, 220, 220, 188]) torch.Size([1, 3, 220, 220, 188])
Dice Score: 0.9032281041145325
torch.Siz

KeyboardInterrupt: 